In [ ]:
!pip install docling langchain langchain-community langchain-openai faiss-cpu

## Fonctions Utils

In [2]:
import requests
import json
import os
import pandas as pd
from langchain_huggingface import HuggingFaceEmbeddings
from docling.document_converter import DocumentConverter, PdfFormatOption
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter, Language
from docling.datamodel.base_models import InputFormat
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.pipeline.vlm_pipeline import VlmPipeline
from docling.datamodel.pipeline_options import VlmPipelineOptions
from docling.datamodel.pipeline_options_vlm_model import ApiVlmOptions, ResponseFormat
from dotenv import load_dotenv
import os
import time
load_dotenv(override=True)
api_key = os.getenv("LLAMA_CLOUD_API_KEY")
os.environ["LLAMA_CLOUD_API_KEY"] = api_key

SERVER = "http://localhost:11434"
MODEL = "mistral"
#VISION_MODEL = "llava"
EMBEDDING_MODEL = "nomic-embed-text"
DB_PATH = "./db_local"

def query_ollama(model: str, prompt: str) -> str:
    """Envoie une requête à Ollama et récupère la réponse"""
    payload = {
        "model": model,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "stream": False
    }
    try:
        r = requests.post(f"{SERVER}/api/chat", json=payload)
        r.raise_for_status()
        response = r.json()
        return response.get("message", {}).get("content", "").strip()
    except requests.exceptions.ConnectionError:
        print("Erreur : Ollama n'est pas lancé")
        return ""
    
def ollama_vlm_options(model: str, prompt: str):
    options = ApiVlmOptions(
        url="http://localhost:11434/v1/chat/completions",
        params=dict(
            model=model,
        ),
        prompt=prompt,
        timeout=120,
        scale=1.0,
        response_format=ResponseFormat.MARKDOWN,
    )
    return options


/home/cytech/Ing3/NLP/Projet_RAG/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Preprocessing

In [ ]:
import nest_asyncio
nest_asyncio.apply()

from llama_parse import LlamaParse

def step1_llamaparse(pdf_path, output_md_path):
    print(f"Envoi du fichier {pdf_path} à LlamaParse...")
    
    # Multimodal pour décrive images
    parser = LlamaParse(
        result_type="markdown",
        premium_mode=True,
        verbose=True
    )

    # Conversion PDF -> Markdown
    documents = parser.load_data(pdf_path)
    
    # LlamaParse renvoie une liste de documents
    full_markdown = "\n\n".join([doc.text for doc in documents])
    
    # Sauvegarde
    os.makedirs(os.path.dirname(output_md_path), exist_ok=True)
    with open(output_md_path, "w", encoding="utf-8") as f:
        f.write(full_markdown)
        
    print(f"Terminé ! Markdown sauvegardé dans : {output_md_path}")
    # Apreçu du markdown
    print(full_markdown[:500])
    return full_markdown

In [4]:
pdf_input = "data/test/raw/2512.02906v2.pdf"
md_output = "data/test/processed/document.md"

os.makedirs("data/test/processed", exist_ok=True)

step1_llamaparse(pdf_input, md_output)

Envoi du fichier data/test/raw/2512.02906v2.pdf à LlamaParse...
Started parsing the file under job_id 770f1b0a-9ec9-424d-9c65-920b996bbc50
Terminé ! Markdown sauvegardé dans : data/test/processed/document.md

# MRD: Multi-resolution Retrieval-Detection Fusion for High-Resolution Image Understanding

| Fan Yang<br/>Harbin Institute of Technology (Shenzhen)<br/>25b951055\@stu.hit.edu.cn | Kaihao Zhang<br/>Harbin Institute of Technology (Shenzhen)<br/>super.khzhang\@gmail.com |
| ------------------------------------------------------------------------------------ | --------------------------------------------------------------------------------------- |


## Abstract

*Understanding high-resolution ima


'\n# MRD: Multi-resolution Retrieval-Detection Fusion for High-Resolution Image Understanding\n\n| Fan Yang<br/>Harbin Institute of Technology (Shenzhen)<br/>25b951055\\@stu.hit.edu.cn | Kaihao Zhang<br/>Harbin Institute of Technology (Shenzhen)<br/>super.khzhang\\@gmail.com |\n| ------------------------------------------------------------------------------------ | --------------------------------------------------------------------------------------- |\n\n\n## Abstract\n\n*Understanding high-resolution images remains a significant challenge for multimodal large language models (MLLMs). Recent study address this issue by dividing the image into smaller crops and computing the semantic similarity between each crop and a query using a pretrained retrieval-augmented generation (RAG) model. The most relevant crops are then selected to localize the target object and suppress irrelevant information. However, such crop-based processing can fragment complete objects across multiple crops, ther

## Extraction des metadonnée

In [5]:
def extract_metadata_with_llm(md_text_start):
    """
    Sends the first 3000 characters of Markdown to Mistral (via local Ollama)
    """

    text_sample = md_text_start[:3000] + "\n\n... [CONTENU IGNORÉ] ...\n\n"
    prompt = """You are an expert librarian assistant.
    Analyze the beginning of this scientific document (Markdown format) and extract the following information in strict JSON format:
    - "title": The complete title of the paper.
    - "authors": The list of authors (separated by commas).
    - "year": The publication year (if found, otherwise "Unknown").
    
    TEXT:
    {text}
    
    Reply ONLY with the JSON, no additional explanations.
    """.format(text=text_sample)
    
    response = query_ollama(MODEL, prompt)
    
    try:
        content = response.strip()
        metadata = json.loads(content)
        if isinstance(metadata.get("authors"), list):
            metadata["authors"] = ", ".join(metadata["authors"])

        return metadata
    except json.JSONDecodeError:
        print(f"JSON Parse Error. Response was: {response[:100]}")
        return {"title": "Unknown", "authors": "Unknown", "year": "Unknown"}

## Convertion markdown en donnée json

In [6]:
import json
import re

def step2_markdown_to_json(md_path, json_output_path):
    print(f"Chargement du Markdown : {md_path}")
    
    with open(md_path, "r", encoding="utf-8") as f:
        md_text = f.read()

    # Extraction métadonnées globales 
    print("Extraction des métadonnées globales via LLM...")
    meta_globales = extract_metadata_with_llm(md_text)
    print(f"   Titré détecté : {meta_globales['title']}")
    print(f"   Auteurs : {meta_globales['authors']}")
    print(f"   Année : {meta_globales['year']}")
    print("Nettoyage de la bibliographie...")
    # Suppression section References
    if "\n# References" in md_text:
        md_text = md_text.split("\n# References")[0]
    elif "\n## References" in md_text:
        md_text = md_text.split("\n## References")[0]
    
    
    markdown_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=[
            ("#", "Titre"),
            ("##", "Section"),
            ("###", "Sous_Section"),
    ]
        )
    md_header_splits = markdown_splitter.split_text(md_text)

    text_splitter = RecursiveCharacterTextSplitter.from_language(
        language=Language.MARKDOWN,
        chunk_size=1000,
        chunk_overlap=200,
        add_start_index=True,

    )
    final_docs = text_splitter.split_documents(md_header_splits)

    json_output = []
    for doc in final_docs:
        chunk_data = {
            "page_content": doc.page_content,
            "metadata": {
                "source": md_path,
                # Récupération automatique des métadonnées du MarkdownSplitter
                "section": doc.metadata.get("Section", "Introduction"),
                "sous_section": doc.metadata.get("Sous_Section", ""),
                "titre": doc.metadata.get("Titre", ""),
                "meta_title": str(meta_globales.get("title", "Unknown")),
                "meta_authors": str(meta_globales.get("authors", "Unknown")),
                "meta_year": str(meta_globales.get("year", "Unknown")),
            }
        }
        json_output.append(chunk_data)

    # Sauvegarde
    with open(json_output_path, "w", encoding="utf-8") as f:
        json.dump(json_output, f, indent=2, ensure_ascii=False)

    print(f"Terminé ! {len(json_output)} chunks prêts pour ChromaDB dans {json_output_path}")

    return json_output

In [7]:
md_int = "data/test/processed/document.md"
json_output = "data/test/processed/chunks_for_rag.json"

os.makedirs("data/test/processed", exist_ok=True)

step2_markdown_to_json(md_int, json_output)

Chargement du Markdown : data/test/processed/document.md
Extraction des métadonnées globales via LLM...
   Titré détecté : MRD: Multi-resolution Retrieval-Detection Fusion for High-Resolution Image Understanding
   Auteurs : Fan Yang, Kaihao Zhang
   Année : Unknown
Nettoyage de la bibliographie...
Terminé ! 63 chunks prêts pour ChromaDB dans data/test/processed/chunks_for_rag.json


[{'page_content': '| Fan Yang<br/>Harbin Institute of Technology (Shenzhen)<br/>25b951055\\@stu.hit.edu.cn | Kaihao Zhang<br/>Harbin Institute of Technology (Shenzhen)<br/>super.khzhang\\@gmail.com |\n| ------------------------------------------------------------------------------------ | --------------------------------------------------------------------------------------- |',
  'metadata': {'source': 'data/test/processed/document.md',
   'section': 'Introduction',
   'sous_section': '',
   'titre': 'MRD: Multi-resolution Retrieval-Detection Fusion for High-Resolution Image Understanding',
   'meta_title': 'MRD: Multi-resolution Retrieval-Detection Fusion for High-Resolution Image Understanding',
   'meta_authors': 'Fan Yang, Kaihao Zhang',
   'meta_year': 'Unknown'}},
 {'page_content': '*Understanding high-resolution images remains a significant challenge for multimodal large language models (MLLMs). Recent study address this issue by dividing the image into smaller crops and comput

## Fonction pour la baseline et RAG

In [8]:
import os
import glob
import json
from tqdm import tqdm

INPUT_DIR = "data/test/raw"
MD_DIR = "data/test/intermediate"
OUTPUT_JSON = "data/test/processed/corpus_final.json"

def run_baseline():
    print(f"Démarrage du pipeline sur : {INPUT_DIR}")

    pdf_files = glob.glob(os.path.join(INPUT_DIR, "*.pdf"))
    print(f"{len(pdf_files)} fichiers PDF trouvés.")

    if not pdf_files:
        print("Aucun PDF trouvé. Vérifie le dossier data/raw !")
        return

    grand_corpus_chunks = []
    os.makedirs(MD_DIR, exist_ok=True)

    for pdf_path in tqdm(pdf_files, desc="Traitement des documents"):
        filename = os.path.basename(pdf_path)
        base_name = os.path.splitext(filename)[0]
        md_output_path = os.path.join(MD_DIR, f"{base_name}.md")

        try:
            if os.path.exists(md_output_path):
                print(f"Markdown existant trouvé pour {filename}, on saute le parsing.")
            else:
                step1_llamaparse(pdf_path, md_output_path)

            temp_json_path = os.path.join(MD_DIR, f"{base_name}.json")
            chunks_du_fichier = step2_markdown_to_json(md_output_path, temp_json_path)
            grand_corpus_chunks.extend(chunks_du_fichier)

        except Exception as e:
            print(f"\nErreur critique sur {filename} : {e}")
            print("   On continue avec le fichier suivant...")
            continue

    print(f"\nSauvegarde du corpus complet ({len(grand_corpus_chunks)} chunks)...")
    os.makedirs(os.path.dirname(OUTPUT_JSON), exist_ok=True)
    with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
        json.dump(grand_corpus_chunks, f, indent=2, ensure_ascii=False)
    print(f"Terminé ! Fichier prêt pour ChromaDB : {OUTPUT_JSON}")


In [9]:
run_baseline()

Démarrage du pipeline sur : data/test/raw
1 fichiers PDF trouvés.


Traitement des documents:   0%|          | 0/1 [00:00<?, ?it/s]

Envoi du fichier data/test/raw/2512.02906v2.pdf à LlamaParse...
Started parsing the file under job_id b1004670-410c-4dfb-a846-77bf7810899b
Terminé ! Markdown sauvegardé dans : data/test/intermediate/2512.02906v2.md

# MRD: Multi-resolution Retrieval-Detection Fusion for High-Resolution Image Understanding

| Fan Yang<br/>Harbin Institute of Technology (Shenzhen)<br/>25b951055\@stu.hit.edu.cn | Kaihao Zhang<br/>Harbin Institute of Technology (Shenzhen)<br/>super.khzhang\@gmail.com |
| ------------------------------------------------------------------------------------ | --------------------------------------------------------------------------------------- |


## Abstract

*Understanding high-resolution ima
Chargement du Markdown : data/test/intermediate/2512.02906v2.md
Extraction des métadonnées globales via LLM...


Traitement des documents: 100%|██████████| 1/1 [00:31<00:00, 31.03s/it]

   Titré détecté : MRD: Multi-resolution Retrieval-Detection Fusion for High-Resolution Image Understanding
   Auteurs : Fan Yang, Kaihao Zhang
   Année : Unknown
Nettoyage de la bibliographie...
Terminé ! 63 chunks prêts pour ChromaDB dans data/test/intermediate/2512.02906v2.json

Sauvegarde du corpus complet (63 chunks)...
Terminé ! Fichier prêt pour ChromaDB : data/test/processed/corpus_final.json


In [10]:
def run_baseline(question):
    start_time = time.time()

    print(f"Baseline (Sans contexte) : '{question}'")
    
    prompt = f"Réponds à cette question de manière concise :\n\n{question}"
    
    response = query_ollama(MODEL, prompt)
    duration = time.time() - start_time

    print("\n Réponse Baseline (Mistral) :")
    print(response)
    print(f"Temps total : {duration:.2f}s")

    return response, duration

In [11]:
question = "Qu'est-ce que la méthode MRD propose ?"
run_baseline(question)

Baseline (Sans contexte) : 'Qu'est-ce que la méthode MRD propose ?'

 Réponse Baseline (Mistral) :
La méthode MRD (Maximal Redundancy Decomposition) est une technique d'analyse multivariée qui permet d'extraire un sous-ensemble optimisé de variables indépendantes les unes des autres pour expliquer le plus possible de la variance totale dans un ensemble de données.
Temps total : 19.18s


("La méthode MRD (Maximal Redundancy Decomposition) est une technique d'analyse multivariée qui permet d'extraire un sous-ensemble optimisé de variables indépendantes les unes des autres pour expliquer le plus possible de la variance totale dans un ensemble de données.",
 19.175695657730103)

In [12]:
import json
import os
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings


import shutil
DB_PATH = "./db_local"
INPUT_JSON = "data/test/processed/chunks_for_rag.json"

def step3_create_vector_db():
    print(f"Chargement des chunks depuis : {INPUT_JSON}")
    print(f"Base de données vectorielle sauvegardée avec succès dans : {DB_PATH}")

    if os.path.exists(DB_PATH):
        print(f"Suppression de l'ancienne DB corrompue...")
        shutil.rmtree(DB_PATH, ignore_errors=True)

    os.makedirs(DB_PATH, exist_ok=True)
    os.chmod(DB_PATH, 0o777)
        
        
    if not os.path.exists(INPUT_JSON):
        print(f"Erreur : Le fichier {INPUT_JSON} n'existe pas. Lance le Step 2 d'abord.")
        return
    

    # Lire JSON
    with open(INPUT_JSON, "r", encoding="utf-8") as f:
        chunks_data = json.load(f)

    # Reconvertir JSON en objets "Document" LangChain
    documents = []
    for item in chunks_data:
        doc = Document(
            page_content=item["page_content"],
            metadata=item["metadata"],
            
        )
        documents.append(doc)
    
    print(f"{len(documents)} documents prêts à être vectorisés.")

    # Embedding
    print(f"Initialisation de l'embedding ({EMBEDDING_MODEL})...")
    embeddings = OllamaEmbeddings(
        model=EMBEDDING_MODEL,
        base_url=SERVER,
    )
    vector = embeddings.embed_query("Test")
    
    print(f"Succes ! Ollama a renvoyé un vecteur de taille {len(vector)}.")
    # DB Chroma
    print("Création des vecteurs et indexation (Cela peut prendre un moment)...")
    vector_db = Chroma.from_documents(
        documents=documents,
        embedding=embeddings,
        persist_directory=DB_PATH,
        collection_name="pdf_rag_collection",
    )
    
    print(f"Base de données vectorielle sauvegardée avec succès dans : {DB_PATH}")


In [13]:
step3_create_vector_db()

Chargement des chunks depuis : data/test/processed/chunks_for_rag.json
Base de données vectorielle sauvegardée avec succès dans : ./db_local
Suppression de l'ancienne DB corrompue...
63 documents prêts à être vectorisés.
Initialisation de l'embedding (nomic-embed-text)...
Succes ! Ollama a renvoyé un vecteur de taille 768.
Création des vecteurs et indexation (Cela peut prendre un moment)...
Base de données vectorielle sauvegardée avec succès dans : ./db_local


In [14]:
def run_rag(question):
    """
    Exécute le RAG pour une question donnée sans boucle interactive.
    """
    start_time = time.time()
    
    embeddings = OllamaEmbeddings(
        model=EMBEDDING_MODEL, 
        base_url=SERVER
    )
    
    vector_db = Chroma(
        embedding_function=embeddings,
        persist_directory=DB_PATH,
        collection_name="pdf_rag_collection",
    )
    
    print(f"RAG sur : '{question}'")

    # Recherche dans la DB
    results = vector_db.similarity_search(question, k=5)
    
    if not results:
        return "Aucune info trouvée dans la base."

    # Construction du contexte
    context_str = ""
    sources_list = []
    
    for doc in results:
        meta = doc.metadata
        titre = meta.get('meta_title', 'Doc')
        section = meta.get('section', 'Section Inconnue')
        source_info = f"{titre} > {section}"
        
        sources_list.append(source_info)
        context_str += f"---\n[Source: {source_info}]\n{doc.page_content}\n"

    # Prompt final
    final_prompt = f"""Tu es un assistant scientifique expert. 
    Utilise EXCLUSIVEMENT le contexte ci-dessous pour répondre à la question.
    Si la réponse contient des éléments visuels (graphiques), décris-les d'après le texte.

    CONTEXTE :
    {context_str}

    QUESTION : 
    {question}

    RÉPONSE :"""

    # Génération
    print("Model : " + MODEL)
    reponse = query_ollama(MODEL, final_prompt)
    
    # Affichage et retour
    print("\n" + reponse)
    
    # Stats
    duration = time.time() - start_time
    print(f"\nSources utilisées ({len(results)} chunks) :")
    for src in set(sources_list):
        print(f"    - {src}")
    print(f"Temps total : {duration:.2f}s")
    
    return reponse, duration, sources_list

In [15]:
question = "Qu'est-ce que la méthode MRD propose ?"
run_rag(question)

RAG sur : 'Qu'est-ce que la méthode MRD propose ?'
Model : mistral

La méthode MRD (Multi-resolution Retrieval-Detection Fusion) est une méthode de fusion de recherche et détection multi-résolution, conçue pour améliorer l'entendement des images à haute résolution par les réseaux de machines à learning multi-tâches (MLLMs).

Elle utilise deux composants principaux : RAG (Retrieval Augmented Graph) et OVD (Object Vertex Detection), qui permettent respectivement d'obtenir une carte de similitude sémantique et une carte de confiance de détection. Par intégration des deux composants, les objets cibles peuvent être localisés plus précisément (Figure 1).

OVD aide à localiser plus précisément un objet unique mais peut entraîner l'omission d'objets dans des scénarios avec plusieurs objets. Le fusion sémantique multi-résolution corrige les scores de similitude sémantique et préserve la complétude des objets sous différentes conditions, améliorant ainsi le rendement de MLLM sur les tâches à un 

("La méthode MRD (Multi-resolution Retrieval-Detection Fusion) est une méthode de fusion de recherche et détection multi-résolution, conçue pour améliorer l'entendement des images à haute résolution par les réseaux de machines à learning multi-tâches (MLLMs).\n\nElle utilise deux composants principaux : RAG (Retrieval Augmented Graph) et OVD (Object Vertex Detection), qui permettent respectivement d'obtenir une carte de similitude sémantique et une carte de confiance de détection. Par intégration des deux composants, les objets cibles peuvent être localisés plus précisément (Figure 1).\n\nOVD aide à localiser plus précisément un objet unique mais peut entraîner l'omission d'objets dans des scénarios avec plusieurs objets. Le fusion sémantique multi-résolution corrige les scores de similitude sémantique et préserve la complétude des objets sous différentes conditions, améliorant ainsi le rendement de MLLM sur les tâches à un objet unique et multiples. Le modèle final, qui intègre tous l

## Reranking

In [16]:
import pandas as pd
import time
from sentence_transformers import CrossEncoder
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from IPython.display import display


from sentence_transformers import CrossEncoder
import time

def run_rag_with_reranking(question):
    print(f"RAG Reranking sur : '{question}'")
    start_time = time.time()
    
    # Retrieval large de documents (10)
    embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL, base_url=SERVER)
    vector_db = Chroma(embedding_function=embeddings, persist_directory=DB_PATH, collection_name="pdf_rag_collection")
    initial_docs = vector_db.similarity_search(question, k=10)
    
    if not initial_docs:
        return "Pas de docs", 0, []

    # Reranking
    reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
    pairs = [[question, doc.page_content] for doc in initial_docs]
    scores = reranker.predict(pairs)
    
    # On trie du meilleur score au pire
    ranked_results = sorted(zip(initial_docs, scores), key=lambda x: x[1], reverse=True)
    
    # Selection du top 3
    top_docs = [doc for doc, score in ranked_results[:3]]
    
    # Generer réponse
    context_str = "\n".join([d.page_content for d in top_docs])
    prompt = f"Tu es un expert. Contexte :\n{context_str}\n\nQuestion : {question}\nRéponse :"
    response = query_ollama(MODEL, prompt)
    
    duration = time.time() - start_time
    
    # Sources formatées
    sources_format = []
    for doc in top_docs:
        meta = doc.metadata
        titre = meta.get('meta_title', meta.get('source', 'Doc'))
        if "/" in titre:
            titre = titre.split("/")[-1]
            
        section = meta.get('section', 'Section Inconnue')
        
        source_info = f"{titre} > {section}"
        sources_format.append(source_info)
    
    return response, duration, sources_format

In [17]:
question = "Qu'est-ce que la méthode MRD propose ?"
run_rag_with_reranking(question)

RAG Reranking sur : 'Qu'est-ce que la méthode MRD propose ?'


("La méthode MRD (Multi-Resolution Diffusion) propose un cadre pour améliorer la compréhension de images à haute résolution par les modèles de langage multimodal (MLLMs). Il intègre différents modules, chacun avec une fonction spécifique :\n\n1. OVD (Object Visual Detection) : Permet d'identifier plus précisément un objet isolé, mais peut manquer des objets dans les scénarios multi-objets.\n2. Multi-resolution semantic fusion : Corrige les scores de similarité sématique et préserve l'intégrité des objets sous différentes conditions, améliorant ainsi le rendement du MLLM sur les tâches uni- et multi-objets.\n3. RAG (Region Attention Generation) : Permet d'attribuer une attention à différentes régions de l'image pour faciliter la compréhension semantique globale.\n4. HR Image : Utilise des images à haute résolution pour améliorer la performance du MLLM.\n\nL'intégration de tous les modules dans le modèle final permet d'atteindre une précision plus élevée de 5,7% par rapport au RAP (Rapid

## Bonus Recherche hybride

In [ ]:
from langchain_community.retrievers import BM25Retriever


def run_rag_hybrid(question):
    """
    RAG Hybride : combine recherche vectorielle et lexicale (BM25) manuellement
    """
    start_time = time.time()
    
    embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL, base_url=SERVER)
    vector_db = Chroma(
        embedding_function=embeddings,
        persist_directory=DB_PATH,
        collection_name="pdf_rag_collection",
    )
    
    print(f"RAG Hybride sur : '{question}'")
    
    all_docs_data = vector_db.get()
    documents = []
    for i, content in enumerate(all_docs_data['documents']):
        doc = Document(
            page_content=content,
            metadata=all_docs_data['metadatas'][i]
        )
        documents.append(doc)
        
    bm25_retriever = BM25Retriever.from_documents(documents)
    bm25_retriever.k = 5
    
    results_bm25 = bm25_retriever.invoke(question)
    results_vector = vector_db.similarity_search(question, k=5)
    
    seen = set()
    results = []
    
    for doc in results_bm25 + results_vector:
        doc_id = doc.page_content[:100]
        if doc_id not in seen:
            seen.add(doc_id)
            results.append(doc)
            if len(results) >= 5:
                break
    
    if not results:
        return "Pas de docs", 0, []
    
    context_str = "\n".join([d.page_content for d in results])
    prompt = f"Tu es un expert. Contexte :\n{context_str}\n\nQuestion : {question}\nReponse :"
    
    reponse = query_ollama(MODEL, prompt)
    duration = time.time() - start_time
    
    sources_list = []
    for doc in results:
        meta = doc.metadata
        titre = meta.get('meta_title', 'Doc')
        if "/" in titre:
            titre = titre.split("/")[-1]
        section = meta.get('section', 'Section Inconnue')
        source_info = f"{titre} > {section}"
        sources_list.append(source_info)
    
    return reponse, duration, sources_list

In [19]:
question = "Qu'est-ce que la méthode MRD propose ?"
run_rag_hybrid(question)

RAG Hybride sur : 'Qu'est-ce que la méthode MRD propose ?'


("La méthode MRD (Multi-resolution Retrieval Detection) propose une méthode sans formation, appelée Multi-resolution Retrieval-Detection, pour améliorer la compréhension des images à haute résolution par les MLLMs (Modèles de langage large pré-entraînés). Elle utilise une similitude semantique multi-résolution pour corriger les cartes de similarité semantique uni-résolution, ce qui garantit l'intégrité des objets cibles. De plus, elle introduit un modèle OVD (Open-vocabulary object) pour identifier les régions d'objets à l'aide d'un approche fenêtre glissante. La méthode MRD améliore significativement la compréhension des images à haute résolution par les MLLMs en comparaison aux approches précédentes et aux modèles de base.",
 140.19953536987305,
 ['MRD: Multi-resolution Retrieval-Detection Fusion for High-Resolution Image Understanding > 6. Conclusion',
  'MRD: Multi-resolution Retrieval-Detection Fusion for High-Resolution Image Understanding > 3. Preliminary',
  'MRD: Multi-resolut

# Test complet

In [ ]:
def demonstration_complete():
    """
    Démonstration complète avec sauvegarde détaillée pour visualisation Streamlit
    Sauvegarde en CSV et JSON pour maximum de flexibilité
    """
    questions = [
        "Qu'est-ce que la méthode MRD propose ?",
        "Quels sont les résultats expérimentaux principaux ?",
        "Quelle est la différence avec les méthodes existantes ?",
        "Quels datasets sont utilisés pour l'évaluation ?",
        "Quelles sont les limitations de l'approche ?"
    ]
    
    results = []
    
    for idx, q in enumerate(questions, 1):
        print(f"QUESTION {idx}/{len(questions)} : {q}")
        
        # Baseline
        print("\n[BASELINE]")
        resp_baseline, time_baseline = run_baseline(q)
        
        # RAG
        print("\n[RAG]")
        resp_rag, time_rag, src_rag = run_rag(q)

        # RAG Hybrid
        print("\n[RAG HYBRID]")
        resp_rag_hybrid, time_rag_hybrid, src_rag_hybrid = run_rag_hybrid(q)
        
        # Reranking
        print("\n[RERANKING]")
        resp_rag_rerank, time_rag_rerank, src_rag_rerank = run_rag_with_reranking(q)
        
        # Stockage complet des résultats
        results.append({
            'id': idx,
            'question': q,
            
            # Baseline
            'baseline_response': resp_baseline,
            'baseline_time': round(time_baseline, 2),
            'baseline_length': len(resp_baseline),
            
            # RAG
            'rag_response': resp_rag,
            'rag_time': round(time_rag, 2),
            'rag_length': len(resp_rag),
            'rag_sources_count': len(src_rag),
            'rag_sources': ' | '.join(src_rag) if src_rag else 'Aucune',

            # RAG Hybrid
            'rag_hybrid_response': resp_rag_hybrid,
            'rag_hybrid_time': round(time_rag_hybrid, 2),
            'rag_hybrid_length': len(resp_rag_hybrid),
            'rag_hybrid_sources_count': len(src_rag_hybrid),
            'rag_hybrid_sources': ' | '.join(src_rag_hybrid) if src_rag_hybrid else 'Aucune',
            
            # Reranking
            'rerank_response': resp_rag_rerank,
            'rerank_time': round(time_rag_rerank, 2),
            'rerank_length': len(resp_rag_rerank),
            'rerank_sources_count': len(src_rag_rerank),
            'rerank_sources': ' | '.join(src_rag_rerank) if src_rag_rerank else 'Aucune',
        })
    
    # Sauvegarde DataFrame
    df = pd.DataFrame(results)
    
    # Sauvegarde CSV (meilleur pour Streamlit - facile à lire)
    df.to_csv('data/test/processed/demo_results.csv', index=False, encoding='utf-8')
    print(f"\nResultats sauvegardes dans data/test/processed/demo_results.csv")
    
    # Sauvegarde JSON (structure pour manipulation programmatique)
    with open('data/test/processed/demo_results.json', 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    print(f"Resultats sauvegardes dans data/test/processed/demo_results.json")
    
    # Sauvegarde Excel (pour lecture humaine facile)
    df.to_excel('data/test/processed/demo_results.xlsx', index=False)
    print(f"Resultats sauvegardes dans data/test/processed/demo_results.xlsx")
    
    # Afficher résumé
    print("\n" + "-"*10)
    print("Resume :")
    print("-"*10)
    print(f"\nTemps moyen :")
    print(f"  Baseline  : {df['baseline_time'].mean():.2f}s")
    print(f"  RAG       : {df['rag_time'].mean():.2f}s")
    print(f"  RAG Hybrid: {df['rag_hybrid_time'].mean():.2f}s")
    print(f"  Reranking : {df['rerank_time'].mean():.2f}s")
    
    print(f"\nLongueur moyenne des reponses :")
    print(f"  Baseline  : {df['baseline_length'].mean():.0f} caracteres")
    print(f"  RAG       : {df['rag_length'].mean():.0f} caracteres")
    print(f"  RAG Hybrid: {df['rag_hybrid_length'].mean():.0f} caracteres")
    print(f"  Reranking : {df['rerank_length'].mean():.0f} caracteres")
    
    print(f"\nSources moyennes :")
    print(f"  RAG       : {df['rag_sources_count'].mean():.1f} sources")
    print(f"  RAG Hybrid: {df['rag_hybrid_sources_count'].mean():.1f} sources")
    print(f"  Reranking : {df['rerank_sources_count'].mean():.1f} sources")
    
    return df

In [21]:
df_final = demonstration_complete()


QUESTION 1/5 : Qu'est-ce que la méthode MRD propose ?

[BASELINE]
Baseline (Sans contexte) : 'Qu'est-ce que la méthode MRD propose ?'

 Réponse Baseline (Mistral) :
La méthode MRD (Mean Residual Disease) est un outil utilisé en oncologie pour évaluer la réponse à un traitement anticancéreux. Il s'agit de mesurer le taux résiduel de maladie dans les cellules tumorales restantes après une première phase de traitement pour déterminer la réponse au traitement (complete response, partial response, stable disease ou progressive disease) et ainsi guider le choix du traitement suivant.
Temps total : 28.08s

[RAG]
RAG sur : 'Qu'est-ce que la méthode MRD propose ?'
Model : mistral

La méthode MRD (Multi-resolution Retrieval-Detection Fusion) propose une méthode de fusion pour l'entendre des images à haute résolution par les réseaux de neurones profonds multi-échelon (MLLMs). Elle utilise deux modules principaux : RAG et OVD. RAG (Retrieval-Augmented Guidance) est utilisé pour obtenir une carte 

In [22]:
df_final

,id,question,baseline_response,baseline_time,baseline_length,rag_response,rag_time,rag_length,rag_sources_count,rag_sources,rag_hybrid_response,rag_hybrid_time,rag_hybrid_length,rag_hybrid_sources_count,rag_hybrid_sources,rerank_response,rerank_time,rerank_length,rerank_sources_count,rerank_sources
0,1,Qu'est-ce que la méthode MRD propose ?,La méthode MRD (Mean Residual Disease) est un ...,28.08,419,La méthode MRD (Multi-resolution Retrieval-Det...,200.27,752,5,MRD: Multi-resolution Retrieval-Detection Fusi...,La méthode MRD (Multi-resolution Retrieval Det...,147.42,789,5,MRD: Multi-resolution Retrieval-Detection Fusi...,Le MRD (Multiscale Resolution Design) est une ...,159.91,996,3,MRD: Multi-resolution Retrieval-Detection Fusi...
1,2,Quels sont les résultats expérimentaux princip...,Les résultats expérimentaux principaux dépende...,39.70,568,Les résultats expérimentaux principaux du trav...,175.34,527,5,MRD: Multi-resolution Retrieval-Detection Fusi...,Les résultats expérimentaux principaux indique...,258.45,1527,5,MRD: Multi-resolution Retrieval-Detection Fusi...,Les résultats expérimentaux principaux indique...,130.08,713,3,MRD: Multi-resolution Retrieval-Detection Fusi...
2,3,Quelle est la différence avec les méthodes exi...,Il est difficile de répondre de manière concis...,63.69,880,La méthode MRD (Multi-resolution Retrieval-Det...,246.48,930,5,MRD: Multi-resolution Retrieval-Detection Fusi...,La methode MRD proposee dans ce travail est un...,156.11,670,5,MRD: Multi-resolution Retrieval-Detection Fusi...,La table montre des résultats de différents mo...,310.82,1911,3,MRD: Multi-resolution Retrieval-Detection Fusi...
3,4,Quels datasets sont utilisés pour l'évaluation ?,Les datasets utilisés pour l'évaluation varien...,30.62,382,Les datasets utilisés pour l'évaluation sont c...,126.30,199,5,MRD: Multi-resolution Retrieval-Detection Fusi...,Les datasets utilisés pour l'évaluation sont V...,140.58,127,5,MRD: Multi-resolution Retrieval-Detection Fusi...,Les données utilisées pour l'évaluation n'appa...,86.32,392,3,MRD: Multi-resolution Retrieval-Detection Fusi...
4,5,Quelles sont les limitations de l'approche ?,L'approche peut être limitée par des facteurs ...,58.78,816,"En utilisant le contexte fourni, l'article pré...",168.09,1013,5,MRD: Multi-resolution Retrieval-Detection Fusi...,"L'approche proposée, Multi-resolution Retrieva...",273.26,1954,5,MRD: Multi-resolution Retrieval-Detection Fusi...,Les limiteurs de l'approche sont les suivants ...,158.35,1515,3,MRD: Multi-resolution Retrieval-Detection Fusi...
